# Introduction

This notebook is summary of Gemini course held by Kaggle.  

First go to [AI Studio](https://aistudio.google.com/app/apikey) and save API KEY locally in a file (`.env`).  

In [ ]:
API_KEY=your_actual_api_key_here

Then you install SDK

In [ ]:
%pip install -U -q "google-generativeai>=0.8.3"

After installing SDK you are ready to load:
- necessary packages 
- load API_KEY

In [1]:
import google.generativeai as genai
from IPython.display import HTML, Markdown, display

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Access the API_KEY environment variable
api_key = os.environ.get('API_KEY')
genai.configure(api_key=api_key)

flash = genai.GenerativeModel(model_name='gemini-1.5-flash')

Now try your first question from Gemini!

In [2]:
flash = genai.GenerativeModel('gemini-1.5-flash')
response = flash.generate_content("Explain AI to me like I'm a kid.")
print(response.text)

Imagine you have a super smart puppy.  You teach it tricks, like "sit" and "fetch".  At first, the puppy doesn't know what those words mean, but you show it, and it learns!

AI is kind of like that super smart puppy, but instead of learning tricks, it learns from information.  We give it lots and lots of information – like pictures of cats and dogs, or stories, or even numbers – and it learns to recognize patterns and make decisions based on that information.

So, if you show an AI lots of pictures of cats, it will learn what a cat looks like and can then tell you if a new picture is a cat or not.  It's not actually *thinking* like you or me, but it's getting really good at following instructions and solving problems using the information it's been given.

Some AIs are simple, like that puppy learning basic tricks. Others are super complex and can do amazing things, like helping doctors diagnose illnesses or recommending your favorite movies!  It's all about learning from information a

You can start a chat by 

In [ ]:
chat = flash.start_chat(history=[])
response = chat.send_message('Hello! My name is Zlork.')
print(response.text)

Check out list of models available in Gemini ([Read more here](https://ai.google.dev/gemini-api/docs/models/gemini))


In [ ]:
for model in genai.list_models():
  print(model.name)

[model overview page](https://ai.google.dev/gemini-api/docs/models/gemini).

# Document Q&A with RAG

LLMs have two limitations:
Knowledge is limited to the trained data: Language models are trained on large datasets but cannot dynamically learn or adapt to new information. This means their knowledge might be incomplete or outdated, especially for niche or rapidly evolving topics.

Input limitations: Models can only respond based on the input provided at the moment, constrained by token limits and lacking the ability to integrate external data dynamically during the interaction.

To address these limitations, Retrieval-Augmented Generation (RAG) combines the power of LLMs with a retrieval mechanism to access and integrate external knowledge dynamically. RAG works in three key steps:

1. Indexing  
The process begins by creating an index of the external knowledge source. This source can include structured or unstructured data such as:
- Documents  
- Databases  
- Research papers  
- Knowledge graphs  
- Websites  
Tools like vector databases (e.g., Qdrant, Pinecone, ChromaDB, or FAISS) are used to convert text into embeddings—a numerical representation of semantic meaning. These embeddings are indexed to enable fast and efficient searches based on relevance.  
2. Retrieval  
When a query is received, the system uses the indexed embeddings to find the most relevant pieces of information from the knowledge base.  
Retrieval involves matching the query with stored embeddings using similarity metrics (e.g., cosine similarity). This step ensures the model has access to contextually appropriate information that may not be part of its trained data.  
Retrieved information is then passed to the language model as supplementary context.
3. Generation  
The language model generates a response by combining its inherent knowledge with the retrieved external data.  
The retrieved content acts as an extension of the model’s training, enhancing its output with up-to-date and specific information tailored to the query.  
This hybrid approach enables the model to produce factually accurate and context-aware responses, addressing the limitations of training data and input constraints.  
Why Use RAG? RAG enhances the utility of LLMs in applications such as:  

- Dynamic question-answering systems  
- Personalized recommendations  
- Interactive data exploration  
- Research and analysis tools  
- Real-time customer support  

This combination of retrieval and generation creates a powerful framework for overcoming the inherent limitations of standalone LLMs.  



We creat embedding by `ChromaDB` model and then use it to generate content by `gemini-1.5-flash` model.


In [3]:
%pip install -U -q "google-generativeai>=0.8.3" chromadb


Note: you may need to restart the kernel to use updated packages.


In [4]:
import google.generativeai as genai
from IPython.display import Markdown

## Data
The data consists of three documents (with text) we create a three variables and a list referring to these variables

In [5]:
DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.'
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]


Let's select `text-embedding-004`.  

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from google.api_core import retry


class GeminiEmbeddingFunction(EmbeddingFunction):
    """ArithmeticError
    Embedding function that uses the Google AI text embedding API to generate embeddings for documents or queries.
    
    Args:
        document_mode: Whether to generate embeddings for documents (True) or queries (False).

    Returns:
        Embeddings: The embeddings generated by the API.

    """
    # Specify whether to generate embeddings for documents, or queries
    document_mode = True

    def __call__(self, input: Documents) -> Embeddings: 
        # Determine the embedding task based on the document_mode
        if self.document_mode:
            embedding_task = "retrieval_document"
        else:
            embedding_task = "retrieval_query"

        # Specify the retry policy for the API request
        # Retry on transient errors
        # See https://googleapis.dev/python/google-api-core/latest/retry.html
        
        retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

        # Generate embeddings using the Google AI text embedding API
        response = genai.embed_content(
            model="models/text-embedding-004",
            content=input,
            task_type=embedding_task,
            request_options=retry_policy,
        )
        return response["embedding"] 

This following code creates a vector database collection named "googlecardb" using ChromaDB. It defines an embedding function, enables document mode, and adds documents to the collection with unique IDs.

In [7]:
import chromadb
# Create a new collection in ChromaDB
DB_NAME = "googlecardb" 
embed_fn = GeminiEmbeddingFunction() # Create an instance of the embedding function
embed_fn.document_mode = True # Set the document_mode to True

chroma_client = chromadb.Client() # Create a new ChromaDB client
db = chroma_client.get_or_create_collection(name=DB_NAME, embedding_function=embed_fn)
# Add the documents to the collection
db.add(documents=documents, ids=[str(i) for i in range(len(documents))])

In [8]:
db.count()

3

## Retrieval 
To search the database we switch to query mode.  

In [ ]:
# Switch to query mode when generating embeddings.
embed_fn.document_mode = False

# Search the Chroma DB using the specified query.
query = "How do you use the touchscreen to play music?"

result = db.query(query_texts=[query], n_results=1)
[[passage]] = result["documents"]

Markdown(passage) # Display the passage


Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

## Augmented Generation: Providing the Answer  

Once you’ve retrieved a relevant passage from the document set (retrieval step), you can create a generation prompt for the Gemini API to generate the final answer. In this example, only one passage was retrieved, but in practice, especially with large datasets, it’s best to retrieve multiple passages and let the Gemini model determine their relevance. It's acceptable if some retrieved passages are not directly related to the question, as the generation step will focus only on the relevant ones.

In [10]:
passage_oneline = passage.replace("\n", " ")
query_oneline = query.replace("\n", " ")

# This prompt is where you can specify any guidance on tone, or what topics the model should stick to, or avoid.
prompt = f"""You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: {query_oneline}
PASSAGE: {passage_oneline}
"""
print(prompt)

You are a helpful and informative bot that answers questions using text from the reference passage included below. 
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. 
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and 
strike a friendly and converstional tone. If the passage is irrelevant to the answer, you may ignore it.

QUESTION: How do you use the touchscreen to play music?
PASSAGE: Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.



## Generating the answer  
Now let's generate an answer by `generate_content` 

In [11]:
model = genai.GenerativeModel("gemini-1.5-flash-latest")
answer = model.generate_content(prompt)
Markdown(answer.text)

To play music on your Googlecar's touchscreen, simply touch the "Music" icon on the main display;  it's that easy!  The touchscreen is the main way you'll interact with many features of your car, including navigation, entertainment (like music), and the climate control settings.
